In [30]:
import os
import time
import random
import platform
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split

# 한글 문제
from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':  # 맥OS
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':  # 윈도우
    path = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system...  sorry~~~')

from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 17,8

import warnings
warnings.filterwarnings(action='ignore')

# 데이터 전처리

#### ECLO만 뽑아내기위한 작업

In [31]:
# ECLO만 뽑아내기 위한 train 호출. (전처리 안된 원본데이터)
train = pd.read_csv('./open/train.csv')
# 전국데이터 호출. (전처리 안된 원본데이터)
countrywide = pd.read_csv('./open/external_open/countrywide_accident.csv')
#전국 데이터와 대구 데이터들의 eclo만 뽑아낸것. 총 row 642,384개
eclo = pd.concat([train['ECLO'], countrywide['ECLO']], sort=False)

#### 요일, 기상, 노면, 유형, 연, 월, 시, 도시, 도로형태로 전처리된 데이터

In [32]:
countrywide_df = pd.read_csv('./open/countrywide_processed.csv')
train_df = pd.read_csv('./open/train_processed.csv')
test_df = pd.read_csv('./open/test_processed.csv')

#### 모든 컬럼 일괄적으로 원핫인 하기위해 데이터타입 통일

In [33]:
# 문자열로 변환하는 함수 정의
def convert_to_str(df, columns):
    for col in columns:
        df[col] = df[col].astype(str)
    return df
# train_df, test_df, countrywide에 대한 문자열 변환
columns_to_convert = ['연', '월', '시간']
train_df = convert_to_str(train_df, columns_to_convert)
test_df = convert_to_str(test_df, columns_to_convert)
countrywide_df = convert_to_str(countrywide_df, columns_to_convert)

#### data라는 변수에 모두 넣어서 한번에 원핫인코딩

In [34]:
#데이터 3개 합쳐
data = pd.concat([train_df, countrywide_df, test_df], sort=False)
data = pd.get_dummies(data)

#### 트레인과 전국데이터 만큼 덜어내기

In [35]:
sum_train_ctw_len = len(train) + len(countrywide)
train_enco = data[:sum_train_ctw_len]
test_enco = data[sum_train_ctw_len:]

- 참고자료 : Catboost의 파라미터 설명
    - iterations:트리의 수 또는 라운드 수
    - depth:각 트리의 최대 깊이
    - learning_rate: 각 트리의 기여도
    - eval_set: 검증 세트를 지정
    - early_stopping_rounds: 검증 세트에서 성능이 향상되지 않으면 얼마나 많은 라운드를 기다릴지를 지정
    - verbose: 학습 과정을 출력할 때의 상세도를 조절(안중요)

# Case1-catboost - 로그변환X, 전국데이터X

- 코드생략
RMSLE: 0.46122

# Case2-catboost - 로그변환O, 전국데이터X

- 코드생략
RMSLE: 0.44702

# Case3-catboost - 로그변환X, 전국데이터O

RMSLE: 0.45516

In [36]:
# X_train은 대구+전국 데이터 원핫인코딩 된 데이터(642,384개), y_train은 eclo만(642,384개) 뽑아낸것. 
X_train = train_enco
y_train = eclo

# train 데이터 분할
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# CatBoost 모델 생성, 학습, 예측
model_cbr = CatBoostRegressor(iterations=1000, depth=10, learning_rate=0.05)
model_cbr.fit(X_train, y_train, eval_set=(X_valid, y_valid), early_stopping_rounds=50, verbose=100)
predictions_cbr = model_cbr.predict(X_valid)

def rmsle(y_true, y_pred):
    return np.sqrt(np.mean(np.square(np.log1p(y_true) - np.log1p(y_pred))))

# 테스트 데이터의 실제 값
y_true = y_valid
# 테스트 데이터의 예측 값
y_pred = predictions_cbr

# RMSLE
rmsle_value = rmsle(y_true, y_pred)
print("RMSLE:", rmsle_value)

0:	learn: 3.5999467	test: 3.5385127	best: 3.5385127 (0)	total: 43.5ms	remaining: 43.5s
100:	learn: 3.5301334	test: 3.4797110	best: 3.4797110 (100)	total: 4.26s	remaining: 37.9s
200:	learn: 3.5148898	test: 3.4758877	best: 3.4758877 (200)	total: 8.53s	remaining: 33.9s
300:	learn: 3.5013647	test: 3.4744999	best: 3.4744733 (292)	total: 12.8s	remaining: 29.7s
400:	learn: 3.4900902	test: 3.4740658	best: 3.4740646 (397)	total: 16.9s	remaining: 25.3s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 3.474022122
bestIteration = 412

Shrink model to first 413 iterations.
RMSLE: 0.4551609149868639


# Case4-catboost - 로그변환O, 전국데이터O (Best)

RMSLE: 0.44126

In [37]:
# X_train, y_train 다시 초기화
X_train = train_enco
y_train = np.log2(eclo)

# 데이터 분할
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# CatBoost 시간 측정 시작
start_time = time.time()

# model_cbr을 로그로 학습시킨 모델로 변경하기
model_cbr.fit(X_train, y_train, eval_set=(X_valid, y_valid), early_stopping_rounds=50, verbose=100)

# 학습 시간 측정 종료
end_time = time.time()
elapsed_time = end_time - start_time
print(f"모델 학습 시간: {elapsed_time} 초")

# 테스트 데이터 예측
predictions_cbr = model_cbr.predict(X_valid)
# 테스트 데이터의 실제 값
y_true = y_valid
# 테스트 데이터의 예측 값
y_pred = predictions_cbr

# 로그상태에서 RMSLE 계산. 하지만 제출할때는 eclo가 로그가 아닌 상태에서 예측해야하니 이결과는 참고용으로..
rmsle_value = rmsle(y_true, y_pred)
print("로그 역변환 전 RMSLE:", rmsle_value)
# 로그 역변환 후 RMSLE 계산
predicted_eclo_back = np.exp2(predictions_cbr)
y_true_back = np.exp2(y_true)
rmsle_value_back = rmsle(y_true_back, predicted_eclo_back)
print("로그역변환 후 최종 RMSLE:", rmsle_value_back)


0:	learn: 0.8138172	test: 0.8128166	best: 0.8128166 (0)	total: 45.5ms	remaining: 45.4s
100:	learn: 0.7979535	test: 0.7986129	best: 0.7986129 (100)	total: 4.31s	remaining: 38.3s
200:	learn: 0.7953523	test: 0.7978957	best: 0.7978957 (200)	total: 8.71s	remaining: 34.6s
300:	learn: 0.7930075	test: 0.7977333	best: 0.7977333 (300)	total: 13.1s	remaining: 30.3s
400:	learn: 0.7906201	test: 0.7977173	best: 0.7977026 (360)	total: 17.4s	remaining: 26.1s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.797702572
bestIteration = 360

Shrink model to first 361 iterations.
모델 학습 시간: 36.18071913719177 초
로그 역변환 전 RMSLE: 0.30745931689717604
로그역변환 후 최종 RMSLE: 0.44126652227007473


In [39]:
predictions_sub = model_cbr.predict(test_enco)
predictions_sub = np.exp2(predictions_sub)

In [40]:
predictions_sub

array([3.86890256, 3.37748962, 4.65321717, ..., 4.18228374, 4.18458789,
       4.24743655])

In [41]:
#기존 제출파일 불러오기 
sub = pd.read_csv('./open/sample_submission.csv') 
# 제출용 파일 만들기
sub['ECLO'] = predictions_sub
submission_df = sub[['ID', 'ECLO']]
# CSV 파일로 저장
submission_df.to_csv('Cat_boost_submission.csv', index=False)